# Deep Convolutional Generative Adversarial Network

The first section of this notebook uses code from the following Tensorflow DCGAN tutorial:
https://www.tensorflow.org/tutorials/generative/dcgan?hl=en

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
# To generate GIFs
!pip install imageio

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

import random

from IPython import display

### Load and prepare the dataset

You will use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
test_images = (test_images - 127.5) / 127.5

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
print(test_labels[:100])
print(test_images.shape)
print(test_labels.shape)

### The Generator



In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [ ]:
generator = make_generator_model()

### The Discriminator


In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()

## Define the loss and optimizers



In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss


In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss


In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Save checkpoints


In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop


In [ ]:
EPOCHS = 200
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
'''Modified : the seed is sparse'''
#seed_0 = tf.random.normal([num_examples_to_generate, noise_dim])
seed_0 = np.load('seed_a.npy')
s = 10
perm_list = np.array([np.random.permutation(seed_0.shape[1]) for i in range(seed_0.shape[0])])
supp = np.array([perm_list[i][:s] for i in range(perm_list.shape[0])])

seed = np.array([[seed_0[j,i]*(i in supp[j,:]) for i in range(noise_dim)] for j in range(seed_0.shape[0])], dtype=float)

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
#@title Original train function
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

**Generate and save images**


In [ ]:
#@title Original generate_images function
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]): # The range is 16 (number of generated images in GIF)
      plt.subplot(4, 4, i+1) # 4 rows, 4 columns, Index i+1
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()


In [ ]:
#@title Modifications #1
'''Run this cell instead of the two previous cells to apply modifications'''

class Training:
  def train(self, dataset, epochs):
    dist_for_plot = np.array([])
    for epoch in range(epochs):
      start = time.time()

      for image_batch in dataset:
        train_step(image_batch)

      # Produce images for the GIF as we go
      display.clear_output(wait=True)
      self.generate_and_save_images(generator,
                              epoch + 1,
                              seed)

      # Save the model every 10 epochs
      if (epoch + 1) % 10 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

      print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

      # Store the mean minimum distance for each epoch
      dist_for_plot = np.append(dist_for_plot, self.mean_min_dist)

    # Generate after the final epoch
    display.clear_output(wait=True)
    self.generate_and_save_images(generator,
                            epochs,
                            seed)

    # Final plot
    plt.plot(range(epochs), dist_for_plot)


  def generate_and_save_images(self, model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]): # The range is 16 (number of generated images in GIF)
        plt.subplot(4, 4, i+1) # 4 rows, 4 columns, Index i+1
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

    # Modifications
    min_dist = np.array([], dtype=float)
    for j in range(predictions.shape[0]):
      dist = np.array([], dtype=float)
      for k in range(test_images.shape[0]):
        dist = np.append(dist, np.linalg.norm(predictions[j, :, :, 0] - test_images[k, :, :, 0]))
      min_dist = np.append(min_dist, np.amin(dist)) # Storing the minimum distance in an array

    self.mean_min_dist = np.mean(min_dist) # Use mean of 16 minimum distances to do 1 plot instead of 16

In [ ]:
#@title Modifications #2
'''Run this cell instead of the two previous cells to apply modifications'''

class Training:
  def train(self, dataset, epochs):
    self.saved_predictions = np.array([], dtype=float)
    for epoch in range(epochs):
      start = time.time()

      for image_batch in dataset:
        train_step(image_batch)

      # Produce images for the GIF as we go
      display.clear_output(wait=True)
      self.generate_and_save_images(generator,
                              epoch + 1,
                              seed)

      # Save the model every 10 epochs
      if (epoch + 1) % 10 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

      print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    self.generate_and_save_images(generator,
                            epochs,
                            seed)


  def generate_and_save_images(self, model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    self.saved_predictions = np.append(self.saved_predictions, predictions)

    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]): # The range is 16 (number of generated images in GIF)
        plt.subplot(4, 4, i+1) # 4 rows, 4 columns, Index i+1
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()


## Train the model


In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
'''Run the following cells to train with modifications'''
t = Training()

In [ ]:
t.train(train_dataset, EPOCHS)

In [ ]:
# Restore the latest checkpoint.
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create a GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

In [ ]:
anim_file = 'dcgan_sparse.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)

If you're working in Colab you can download the animation with the code below:

In [ ]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(anim_file)
  #files.download(checkpoint_dir)

In [ ]:
!zip -r /content/file.zip /content/training_checkpoints/

In [ ]:
from google.colab import files
files.download("/content/file.zip")

# Further modifications for research project

In [ ]:
print(t.saved_predictions.shape)

In [ ]:
t.saved_predictions = t.saved_predictions.reshape((EPOCHS+1, num_examples_to_generate,28,28,1))
print(t.saved_predictions.shape)

For each generated image, show the convergence every 5 epochs


In [ ]:
min_dist = np.array([[]], dtype=float)
labels = np.array([], dtype=int) # To get the labels of the minimizers
index_min = np.array([], dtype=int) # To get the exact positions of the minimizers in the dataset
for round in range(t.saved_predictions.shape[0] - 1):
      if (round + 1) % 5 == 0:
        for j in range(t.saved_predictions.shape[1]):
          dist = np.array([], dtype=float)
          for k in range(test_images.shape[0]):
            dist = np.append(dist, np.linalg.norm(t.saved_predictions[round, j, :, :, 0] - test_images[k, :, :, 0]))
          labels = np.append(labels, test_labels[np.argmin(dist)])
          index_min = np.append(index_min, np.argmin(dist))
          min_dist = np.append(min_dist, np.amin(dist))
min_dist = min_dist.reshape((int(EPOCHS/5), num_examples_to_generate))

In [ ]:
np.save('sparse_predictions_a.npy', t.saved_predictions)
np.save('sparse_seed_a.npy', seed)
np.save('sparse_mindist_a.npy', min_dist)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(t.saved_predictions.shape[1]):
  plt.subplot(4, 4, i+1, ylim=(min(min_dist.flatten()),max(min_dist.flatten())))
  plt.plot([5*i+5 for i in range(int(EPOCHS/5))], min_dist[:,i])
plt.show()

In [ ]:
labels = labels.reshape((int(EPOCHS/5),16))
print(labels[-1,:]) # The numbers/labels to which the generated images are closest to
display_image(EPOCHS)

Plot 4x4 grid of minimizers at final epoch

In [ ]:
print(index_min)
print(index_min.shape)

In [ ]:
index_min = index_min.reshape((int(EPOCHS/5),16))
print(index_min[-1,:])

In [ ]:
plt.figure(figsize=(8,8))

for i in range(len(index_min[-1,:])):
  plt.subplot(4, 4, i+1)
  plt.imshow(test_images[index_min[-1,i],:,:,0], cmap='gray')
  plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))

for i in range(t.saved_predictions.shape[1]):
  plt.subplot(4, 4, i+1)
  plt.imshow(t.saved_predictions[-1,i,:,:,0], cmap='gray')
  plt.axis('off')
plt.show()

2D sketches of distances between generated images

In [ ]:
phi = tf.random.normal([2,noise_dim])
print(phi.shape)

In [ ]:
# Get 2D vectors for each generated image from the seed matrix

generated_pts = np.array([[]], dtype=float)

for i in range(num_examples_to_generate):
  generated_pts = np.append(generated_pts, np.matmul(phi,seed[i,:]))

generated_pts = generated_pts.reshape((num_examples_to_generate, 2))
generated_pts = generated_pts.transpose()

lbl = np.array([str(i) for i in labels[-1,:]])

In [ ]:
plt.plot(generated_pts[0], generated_pts[1], 'o')

In [ ]:
mymarkers = ['ro', 'bo', 'b.', 'rx', 'b+', 'bv', 'b,', 'b<', 'b>', 'bs']
for j in range(16):
  plt.plot(generated_pts[0][j], generated_pts[1][j], mymarkers[int(lbl[j])])
plt.legend([str(i) for i in labels[-1,:]], bbox_to_anchor=(1.3, 1))

In [ ]:
# Visualize what is happening between two latent vectors (segment formed by z_1 and z_2)

r_index = random.sample(range(seed.shape[0]),2)
print(r_index)

In [ ]:
z_1 = np.array(seed[r_index[0],:]).reshape((1,100))
plt.imshow(generator(z_1)[0,:,:,0], cmap='gray')

In [ ]:
z_2 = np.array(seed[r_index[1],:]).reshape((1,100))
plt.imshow(generator(z_2)[0,:,:,0], cmap='gray')

In [ ]:
step = 1/100
lbd = 0

plt.figure(figsize=(10,10))
for i in range(int(1/step)):
  next_image = (1-lbd)*z_1 + lbd*z_2
  plt.subplot(10, 10, i+1)
  plt.imshow(generator(next_image)[0,:,:,0], cmap='gray')
  lbd += step
  plt.axis('off')
plt.show()

# Experimenting with sparse vectors

What happens when the latent random vector is sparse ?

In [ ]:
p_list = np.random.permutation(seed.shape[1])
s = 10
support = p_list[:s]
print(support)

In [ ]:
seed_2 = tf.random.normal([1, noise_dim])
sparse_seed = np.array([seed_2[0,i]*(i in support) for i in range(noise_dim)], dtype=float)

sparse_seed = sparse_seed.reshape((1,noise_dim))

In [ ]:
# Random Gaussian latent vector (not sparse)
plt.imshow(generator(seed_2)[0, :, :, 0], cmap='gray')

In [ ]:
# Random s-sparse latent vector
plt.imshow(generator(sparse_seed)[0, :, :, 0], cmap='gray')

Now with multiple latent vectors and their corresponding sparse vector :

In [ ]:
seed_3 = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
p_list_2 = np.array([np.random.permutation(seed_3.shape[1]) for i in range(seed_3.shape[0])])

In [ ]:
support_2 = np.array([p_list_2[i][:s] for i in range(p_list_2.shape[0])])

In [ ]:
sparse_seed_2 = np.array([[seed_3[j,i]*(i in support_2[j,:]) for i in range(noise_dim)] for j in range(seed_3.shape[0])], dtype=float)

In [ ]:
plt.figure(figsize=(5,5))
for i in range(seed_3.shape[0]):
  plt.subplot(4, 4, i+1)
  plt.imshow(generator(seed_3)[i, :, :, 0], cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
for i in range(sparse_seed_2.shape[0]):
  plt.subplot(4, 4, i+1)
  plt.imshow(generator(sparse_seed_2)[i, :, :, 0], cmap='gray')
plt.show()

# Training a generator with a sparse input vector

In [ ]:
generator_2 = make_generator_model()
discriminator_2 = make_discriminator_model()

plt.imshow(generator_2(sparse_seed, training=False)[0, :, :, 0], cmap='gray')

The generated image is roughly the same as when the input vector wasn't sparse